In [1]:
import sys
sys.path.append(r"C:\Users\sarwj\OneDrive - Cardiff University\Documents\GitHub\topologicpy\src") # change the path
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly

In [19]:
import numpy as np

# A class for Axis-Aligned Bounding Box (AABB)
class AABB:
    def __init__(self, min_point, max_point):
        self.min_point = np.array(min_point)
        self.max_point = np.array(max_point)
        self.centroid = (self.min_point + self.max_point) / 2.0

    def intersects(self, other):
        # Check if this AABB intersects with another AABB
        return np.all(self.min_point <= other.max_point) and np.all(self.max_point >= other.min_point)

    def contains(self, point):
        # Check if a point is contained within the AABB
        return np.all(self.min_point <= point) and np.all(self.max_point >= point)

# MeshObject class that stores a reference to the Topologic object
class MeshObject:
    def __init__(self, vertices, topologic_object):
        self.vertices = np.array(vertices)
        self.aabb = AABB(np.min(vertices, axis=0), np.max(vertices, axis=0))
        self.centroid = np.mean(vertices, axis=0)
        self.topologic_object = topologic_object  # Store the Topologic object reference

# BVH Node class
class BVHNode:
    def __init__(self, aabb, left=None, right=None, objects=None):
        self.aabb = aabb
        self.left = left
        self.right = right
        self.objects = objects if objects else []

# Recursive BVH construction
def build_bvh(objects, depth=0):
    if len(objects) == 1:
        return BVHNode(objects[0].aabb, objects=objects)
    
    # Split objects along the median axis based on their centroids
    axis = depth % 3
    objects.sort(key=lambda obj: obj.centroid[axis])
    
    mid = len(objects) // 2
    left_bvh = build_bvh(objects[:mid], depth + 1)
    right_bvh = build_bvh(objects[mid:], depth + 1)

    # Merge left and right bounding boxes
    combined_aabb = AABB(
        np.minimum(left_bvh.aabb.min_point, right_bvh.aabb.min_point),
        np.maximum(left_bvh.aabb.max_point, right_bvh.aabb.max_point)
    )
    
    return BVHNode(combined_aabb, left_bvh, right_bvh)

# Function to perform clash detection (broad-phase) and return Topologic objects
def clash_detection(bvh_node, query_aabb, clashing_objects=None):
    if clashing_objects is None:
        clashing_objects = []
    
    # Check if the query AABB intersects with the current node's AABB
    if not bvh_node.aabb.intersects(query_aabb):
        return clashing_objects
    
    # If this is a leaf node, check each object in the node
    if bvh_node.objects:
        for obj in bvh_node.objects:
            if obj.aabb.intersects(query_aabb):
                clashing_objects.append(obj.topologic_object)  # Return the Topologic object
        return clashing_objects
    
    # Recursively check the left and right child nodes
    clash_detection(bvh_node.left, query_aabb, clashing_objects)
    clash_detection(bvh_node.right, query_aabb, clashing_objects)
    
    return clashing_objects

# Function to recursively add nodes and edges to the TopologicPy Graph
def add_bvh_to_graph(bvh_node, graph, parent_vertex=None, tolerance=0.0001):
    # Create a vertex for the current node's AABB centroid
    centroid = bvh_node.aabb.centroid
    current_vertex = Vertex.ByCoordinates(x=centroid[0], y=centroid[1], z=centroid[2])

    # Add an edge from the parent to this vertex (if a parent exists)
    if parent_vertex is not None:
        d = Vertex.Distance(parent_vertex, current_vertex)
        if d < tolerance:
            current_vertex = Topology.Translate(current_vertex, tolerance*100, tolerance*100, tolerance*100)
        edge = Edge.ByVertices(parent_vertex, current_vertex, tolerance=tolerance)
        graph = Graph.AddEdge(graph, edge)
    
    # Recursively add child nodes
    if bvh_node.left:
        graph = add_bvh_to_graph(bvh_node.left, graph, current_vertex)
    if bvh_node.right:
        graph = add_bvh_to_graph(bvh_node.right, graph, current_vertex)

    return graph

# Example usage


cc = CellComplex.Prism(width=10, length=5, height=4, uSides=3, vSides=3, wSides=3)
exploded = Topology.Explode(cc, scale=2)
cells = Topology.Cells(exploded)
mesh_objects = []
for cell in cells:
    vertices = [(Vertex.X(v), Vertex.Y(v), Vertex.Z(v)) for v in Topology.Vertices(cell)]
    mesh_objects.append(MeshObject(vertices, cell))

bvh_root = build_bvh(mesh_objects)
query_aabb = AABB(min_point=(0, 0, 0), max_point=(0,0,0))

clashing_cluster = Cluster.ByTopologies([Vertex.Origin()])
clashing_objects = clash_detection(bvh_root, query_aabb)
if len(clashing_objects) > 0:
    clashing_cluster = Cluster.ByTopologies(clashing_objects)

print(clashing_objects)
#print("Clash Detected:", clash_detection(bvh_root, query_aabb))

# Create an empty TopologicPy Graph
graph = Graph.ByVerticesEdges([Vertex.Origin()], [])
print(graph)

# Add BVH tree to the TopologicPy Graph
graph = add_bvh_to_graph(bvh_root, graph)
data01= Plotly.DataByGraph(graph, vertexSize=8, vertexColor="red", edgeWidth=4, edgeColor="blue")
data02 = Plotly.DataByTopology(exploded, faceOpacity=0.1)
data03 = Plotly.DataByTopology(clashing_cluster, faceOpacity=0.1, faceColor="red")

figure = Plotly.FigureByData(data01+data02+data03, width=1024, height=960)
Plotly.Show(figure)


In [18]:
import numpy as np

class BVH:
    # A class for Axis-Aligned Bounding Box (AABB)
    class AABB:
        def __init__(self, min_point, max_point):
            self.min_point = np.array(min_point)
            self.max_point = np.array(max_point)
            self.centroid = (self.min_point + self.max_point) / 2.0

        def intersects(self, other):
            # Check if this AABB intersects with another AABB
            return np.all(self.min_point <= other.max_point) and np.all(self.max_point >= other.min_point)

        def contains(self, point):
            # Check if a point is contained within the AABB
            return np.all(self.min_point <= point) and np.all(self.max_point >= point)

    # MeshObject class that stores a reference to the Topologic object
    class MeshObject:
        def __init__(self, vertices, topologic_object):
            self.vertices = np.array(vertices)
            self.aabb = BVH.AABB(np.min(vertices, axis=0), np.max(vertices, axis=0))
            self.centroid = np.mean(vertices, axis=0)
            self.topologic_object = topologic_object  # Store the Topologic object reference

    # BVH Node class
    class BVHNode:
        def __init__(self, aabb, left=None, right=None, objects=None):
            self.aabb = aabb
            self.left = left
            self.right = right
            self.objects = objects if objects else []

    def ByTopologies(topologies):
        if not isinstance(topologies, list):
            topologies = [topologies]
        # Recursive BVH construction
        def build_bvh(objects, depth=0):
            if len(objects) == 1:
                return BVH.BVHNode(objects[0].aabb, objects=objects)
            
            # Split objects along the median axis based on their centroids
            axis = depth % 3
            objects.sort(key=lambda obj: obj.centroid[axis])
            
            mid = len(objects) // 2
            left_bvh = build_bvh(objects[:mid], depth + 1)
            right_bvh = build_bvh(objects[mid:], depth + 1)

            # Merge left and right bounding boxes
            combined_aabb = BVH.AABB(
                np.minimum(left_bvh.aabb.min_point, right_bvh.aabb.min_point),
                np.maximum(left_bvh.aabb.max_point, right_bvh.aabb.max_point)
            )
            
            return BVH.BVHNode(combined_aabb, left_bvh, right_bvh)
        
        mesh_objects = []
        for topology in topologies:
            vertices = [(Vertex.X(v), Vertex.Y(v), Vertex.Z(v)) for v in Topology.Vertices(topology)]
            mesh_objects.append(BVH.MeshObject(vertices, topology))
        
        return build_bvh(mesh_objects)
    
    def QueryByTopologies(topologies):
        if not isinstance(topologies, list):
            topologies = [topologies]
        vertices = []
        for topology in topologies:
            if Topology.IsInstance(topology, "Vertex"):
                vertices.append(topology)
            else:
                vertices.extend(Topology.Vertices(topology))
        cluster = Cluster.ByTopologies(vertices)
        bb = Topology.BoundingBox(cluster)
        d = Topology.Dictionary(bb)
        min_x = Dictionary.ValueAtKey(d, "minx")
        min_y = Dictionary.ValueAtKey(d, "miny")
        min_z = Dictionary.ValueAtKey(d, "minz")
        max_x = Dictionary.ValueAtKey(d, "maxx")
        max_y = Dictionary.ValueAtKey(d, "maxy")
        max_z = Dictionary.ValueAtKey(d, "maxz")
        query_aabb = BVH.AABB(min_point=(min_x, min_y, min_z), max_point=(max_x,max_y,max_z))
        return query_aabb
    
    def Clashes(bvh, query):
        # Function to perform clash detection (broad-phase) and return Topologic objects
        def clash_detection(bvh_node, query_aabb, clashing_objects=None):
            if clashing_objects is None:
                clashing_objects = []
            
            # Check if the query AABB intersects with the current node's AABB
            if not bvh_node.aabb.intersects(query_aabb):
                return clashing_objects
            
            # If this is a leaf node, check each object in the node
            if bvh_node.objects:
                for obj in bvh_node.objects:
                    if obj.aabb.intersects(query_aabb):
                        clashing_objects.append(obj.topologic_object)  # Return the Topologic object
                return clashing_objects
            
            # Recursively check the left and right child nodes
            clash_detection(bvh_node.left, query_aabb, clashing_objects)
            clash_detection(bvh_node.right, query_aabb, clashing_objects)
            
            return clashing_objects
        return clash_detection(bvh, query)

    # Function to recursively add nodes and edges to the TopologicPy Graph
    def Graph(bvh, tolerance=0.0001):
        import random
        def add_bvh_to_graph(bvh_node, graph, parent_vertex=None, tolerance=0.0001):
            # Create a vertex for the current node's AABB centroid
            centroid = bvh_node.aabb.centroid
            current_vertex = Vertex.ByCoordinates(x=centroid[0], y=centroid[1], z=centroid[2])

            # Add an edge from the parent to this vertex (if a parent exists)
            if parent_vertex is not None:
                d = Vertex.Distance(parent_vertex, current_vertex)
                if d < tolerance:
                    current_vertex = Topology.Translate(current_vertex, tolerance*random.uniform(2,50), tolerance*random.uniform(2,50), tolerance*random.uniform(2,50))
                edge = Edge.ByVertices(parent_vertex, current_vertex, tolerance=tolerance)
                graph = Graph.AddEdge(graph, edge, silent=True)
            
            # Recursively add child nodes
            if bvh_node.left:
                graph = add_bvh_to_graph(bvh_node.left, graph, parent_vertex=current_vertex, tolerance=tolerance)
            if bvh_node.right:
                graph = add_bvh_to_graph(bvh_node.right, graph, parent_vertex=current_vertex, tolerance=tolerance)

            return graph
        graph = Graph.ByVerticesEdges([Vertex.Origin()], [])
        return add_bvh_to_graph(bvh, graph, parent_vertex = None, tolerance=tolerance)

# Example usage


cc = CellComplex.Prism(width=10, length=5, height=4, uSides=5, vSides=5, wSides=5)
exploded = Topology.Explode(cc, scale=2)
bounding_box = Topology.BoundingBox(exploded)
d = Topology.Dictionary(bounding_box)
min_x = Dictionary.ValueAtKey(d, "minx")
min_y = Dictionary.ValueAtKey(d, "miny")
min_z = Dictionary.ValueAtKey(d, "minz")
max_x = Dictionary.ValueAtKey(d, "maxx")
max_y = Dictionary.ValueAtKey(d, "maxy")
max_z = Dictionary.ValueAtKey(d, "maxz")
sv = Vertex.ByCoordinates(min_x, min_y, min_z)
ev = Vertex.ByCoordinates(max_x, max_y, max_z)
exploded_cells = Topology.Cells(exploded)
bvh = BVH.ByTopologies(exploded_cells)
graph = BVH.Graph(bvh)
sv = Graph.NearestVertex(graph, sv)
ev = Graph.NearestVertex(graph, ev)
shortest_path = Graph.ShortestPath(graph, sv, ev)
Topology.Show(shortest_path, exploded_cells)
# steps = 2
# for i in range(0,steps+1,1):
#     origin = Wire.VertexByParameter(shortest_path, i/(steps))
#     cells = [Cell.Prism(origin = origin, width=2, length=2, height=2)]
    
#     query = BVH.QueryByTopologies(cells)
#     clashing_cluster = Cluster.ByTopologies([Vertex.Origin()])
#     clashing_objects = BVH.Clashes(bvh, query)
#     if len(clashing_objects) > 0:
#         clashing_cluster = Cluster.ByTopologies(clashing_objects)

    

#     # Add BVH tree to the TopologicPy Graph
#     # graph = add_bvh_to_graph(bvh_root, graph)
#     data00 = Plotly.DataByTopology(Cluster.ByTopologies(cells), faceOpacity=0.1, faceColor="red")
#     data01= Plotly.DataByGraph(graph, vertexSize=8, vertexColor="red", edgeWidth=4, edgeColor="blue")
#     data02 = Plotly.DataByTopology(exploded, faceOpacity=0.1)
#     data03 = Plotly.DataByTopology(clashing_cluster, faceOpacity=1, faceColor="yellow", vertexSize=8)

#     figure = Plotly.FigureByData(data00+data01+data02+data03, width=1024, height=960)
#     status = Plotly.FigureExportToPNG(figure, "C:/Users/sarwj/Downloads/bvh"+str(i).zfill(3)+".png", overwrite=True)
